In [5]:
import os
import re
import torch
from torch.utils.data import Dataset, DataLoader, random_split

from data_loader_v1 import create_dataloader_v1
from clean_gutenberg_text import clean_gutenberg_text

In [6]:
with open("original_texts/Rilke_Neue_Gedichte_aus_Gutenberg.txt", "r", encoding="utf-8") as file:
    text_data = file.read()

with open("original_texts/Rilke_Buch_der_Bilder_Gutenberg.txt", "r", encoding="utf-8") as file:
    text_data += "\n" + file.read()  # Append with a newline separator    

with open("original_texts/Rilke_Stundenbuch_Gutenberg.txt", "r", encoding="utf-8") as file:
    text_data += "\n" + file.read()  # Append with a newline separator  

with open("original_texts/Rilke_Erste_Gedichte_Gutenberg.txt", "r", encoding="utf-8") as file:
    text_data += "\n" + file.read()  # Append with a newline separator    

with open("original_texts/Rilke_Der_Neuen_Gedichte_Anderer_Teil_Gutenberg.txt", "r", encoding="utf-8") as file:
    text_data += "\n" + file.read()  # Append with a newline separator    

In [7]:
# Train/validation ratio
train_ratio = 0.80
split_idx = int(train_ratio * len(text_data))
train_data = text_data[:split_idx]
val_data = text_data[split_idx:]

with open('train_text_data.txt', 'w') as f:
    f.write(train_data)
with open('val_text_data.txt', 'w') as f:
    f.write(val_data)